## Implement an E=3 RSA Broadcast attack

Assume you're a Javascript programmer. That is, you're using a naive handrolled RSA to encrypt without padding.

Assume you can be coerced into encrypting the same plaintext three times, under three different public keys. You can; it's happened.

Then an attacker can trivially decrypt your message, by:

    Capturing any 3 of the ciphertexts and their corresponding pubkeys
    Using the CRT to solve for the number represented by the three ciphertexts (which are residues mod their respective pubkeys)
    Taking the cube root of the resulting number

The CRT says you can take any number and represent it as the combination of a series of residues mod a series of moduli. In the three-residue case, you have: 

```
result =
  (c_0 * m_s_0 * invmod(m_s_0, n_0)) +
  (c_1 * m_s_1 * invmod(m_s_1, n_1)) +
  (c_2 * m_s_2 * invmod(m_s_2, n_2)) mod N_012
```

where:

```
 c_0, c_1, c_2 are the three respective residues mod
 n_0, n_1, n_2

 m_s_n (for n in 0, 1, 2) are the product of the moduli
 EXCEPT n_n --- ie, m_s_1 is n_0 * n_2

 N_012 is the product of all three moduli
```

To decrypt RSA using a simple cube root, leave off the final modulus operation; just take the raw accumulated result and cube-root it. 

In [6]:
from sympy.ntheory.modular import crt

Quick refresher on CRT.

In [2]:
moduli = [99, 97, 95]
residues = [49, 76, 65]

In [3]:
crt(moduli, residues)

(639985, 912285)

In [4]:
[639985 % m for m in moduli]

[49, 76, 65]

Read captured ciphertext and public keys.

In [3]:
n = []
c = []
with open('challenge_40-captured.txt', 'r') as f:
    lines = f.readlines()
    assert len(lines) == 3
    for line in lines:
        tokens = line.split()
        c.append(int(tokens[0]))
        n.append(int(tokens[1]))

In [5]:
print(n)
print(c)

[121301893263996127770714256263889220947, 175764422219872672961029155615129970063, 185406513322745209560535444884415549501]
[92216054062610561908720555277938879121, 150351448251373617373436132616541452444, 173455060021778679598924451004122691418]


In [8]:
x, _ = crt(n, c)
x

7722709502790459166639993213708833737796147876436427049309008419119949153843426449996193173565875000

In [10]:
from sympy import integer_nthroot
from Crypto.Util.number import long_to_bytes

In [13]:
m = integer_nthroot(x, 3)
m

(1976620216402300889624482718775150, True)

In [14]:
long_to_bytes(m[0])

b'attack at dawn'